In [40]:
import numpy as np
import imutils
import cv2

image = cv2.imread("pos_3.jpg",0)
template = cv2.imread("template.png",0)

""" edge detection in the template using canny detector """
template = cv2.Canny(template, 150, 255)
(tH, tW) = template.shape[:2]
print(image.shape)
print(template.shape)
#cv2.imshow("image_initial",image)
#cv2.imshow("template",template)
found=None


pixels=[[0,0,0],[0,0,0]]

for scale in np.linspace(0.1, 3.0,200)[::-1]:
    """
    we will resize the image according to the scale, and keep track
    of the ratio of the resizing so that later on we can multiply that
    ratio with the founded coordinates and get back the position in the
    original image to draw rectangle.
    
    Also, we resize the image because cursor in the image & cursor template
    may not have the same dimensions. So, to make sure that they overlap well
    on each other we try to cover range of scale values to resize the image.
    
    """
    #resized = cv2.resize(image, (0,0), fx=image.shape[0], fy=int(image.shape[1] * scale))
    resized = imutils.resize(image, width = int(image.shape[1] * scale))
    r = image.shape[1] / float(resized.shape[1])
 
    # if the resized image is smaller than the template, then we break
    # from the loop
    if resized.shape[0] < tH or resized.shape[1] < tW:
        break
    
    """
    once the image is resized, we try to detect edges in both template and 
    the image because in template matching, images with their edges detected
    performs better and gives good results. There are many edge detectors 
    available. I have used canny edge detector.
    
    """
    edged = cv2.Canny(resized, 150, 255)
    result = cv2.matchTemplate(edged, template, cv2.TM_CCORR_NORMED)
    (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
    #print(maxVal[0])
    """    
    if we have found a new maximum correlation value, then we will update
    the bookkeeping variable.
    
    """
    
    """
    Here, there is an issue of false cursoer being detected on the side bar.
    So, I am trying to find two different co-ordinates where the correlation value 
    is the highest and  the second highest because when there are two cursors
    in an image, difference between the correlation value at both coordinates(cursors)
    will be minute.
    
    """
    maxima=[maxVal,maxLoc,r]
    if(pixels[0][0]>pixels[1][0]):
        mini = pixels[1]
    else:
        mini = pixels[0]
        
    if(maxima[0]>mini[0]):  
        item=pixels.index(mini)
        del pixels[item]
        pixels.insert(0,maxima)
    
print(pixels)    
        
for i in range(len(pixels)):           
    (_, maxLoc, r) = tuple(pixels[i])

    """
    we are determining the coordinates where the correlation value is highest in
    the resized image. Then, using resize ratio we will determine the corresponding 
    coordinates in the original image.

    """
    (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))

    """
    After that we are using template's dimensions to know the dimensions of the rectangle
    we will be drawing to depict the detected cursor.

    """
    (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))

    # draw a bounding box around the detected result and display the image
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
            
            
#cv2.imshow("temp",template)
#cv2.imshow("edged",edged)
cv2.imshow("Image", image)
cv2.waitKey(100000)    
cv2.destroyAllWindows()

(720, 1280)
(39, 29)
[[0.6611860990524292, (20, 961), 0.505130228887135], [0.6613970398902893, (21, 996), 0.48724781119147315]]
